# This is a learning project. It aims to have hands on experience in NLP

### Trying to build a model that predicts which Product do the customer complaints concern
### Will work with [Customer Complaint Database](https://www.kaggle.com/datasets/selener/consumer-complaint-database) from Kaggle